In [1]:
import os
import random

from cities.queries.causal_insight import CausalInsight
from cities.utils.data_grabber import (DataGrabber, list_interventions,
                                       list_outcomes)

smoke_test = "CI" in os.environ
num_samples = 10 if smoke_test else 1000

In [26]:
#sort and save sorted interventions

from cities.utils.data_grabber import DataGrabber
import matplotlib.pyplot as plt
import numpy as np


interventions = list_interventions()
dg = DataGrabber()

dg.get_features_std_wide(interventions)

interventions_sorted = {}
for intervention in interventions:
    intervention_frame = dg.std_wide[intervention].copy().iloc[:, 2:]
    intervention_frame = intervention_frame.apply(lambda col: col.sort_values().values)
    assert (all(np.diff(intervention_frame[col]) >= 0) for col in intervention_frame.columns), "A column is not increasing."
    
    
        
    

